In [17]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd
import skforecast

print(skforecast.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/joaquin/Documents/GitHub/skforecast
0.21.0


In [18]:
lags_contiguous = np.arange(1, 25)          # [1..24]
lags_noncontig  = np.array([1, 2, 3, 6, 12, 24])

max_lag     = 24
window_size = max_lag   # no window features: window_size == max_lag

rng = np.random.default_rng(42)
y = rng.standard_normal(100_000)
y_strided = np.lib.stride_tricks.sliding_window_view(y, window_size)[:-1]

print(f"y_strided shape: {y_strided.shape}")

y_strided shape: (99976, 24)


In [19]:
# ── Correctness check ──────────────────────────────────────────────────────────
# Both methods must produce the same matrix for contiguous lags.

result_slice = y_strided[:, window_size - max_lag:][:, ::-1]  # view
result_fancy = y_strided[:, window_size - lags_contiguous]     # copy

assert np.array_equal(result_slice, result_fancy), "Results differ!"
print("Correctness check passed: both methods produce identical matrices.")
print(f"result_slice shares memory with y_strided : {np.shares_memory(result_slice, y_strided)}")
print(f"result_fancy shares memory with y_strided : {np.shares_memory(result_fancy, y_strided)}")

Correctness check passed: both methods produce identical matrices.
result_slice shares memory with y_strided : True
result_fancy shares memory with y_strided : False


In [20]:
%%timeit
# Contiguous lags 1-24 — slice + reverse (zero-copy view)
X_data = y_strided[:, window_size - max_lag:][:, ::-1]

704 ns ± 58.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [21]:
%%timeit
# Contiguous lags 1-24 — fancy index (forces a copy)
X_data = y_strided[:, window_size - lags_contiguous]

1.7 ms ± 94.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
%%timeit
# Non-contiguous lags [1,2,3,6,12,24] — fancy index (copy, fewer columns)
X_data = y_strided[:, window_size - lags_noncontig]